<a href="https://colab.research.google.com/github/rmhm1/3654/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: It's an Avocado... Thanks!
# Team Name: Free shAvacado
##### Team Members: Shannon Aikens, Paolo Fermin, Ryland Hanson, Somya Jain 
##### PIDs: shannonaikens, rmhm1, somyajain17, paolofermin

# Topic: Avocados and their Relation to the Economy
#### Description:
Millenials and Generation Z are often criticized for spending their income on unnecessary items. This can include expensive food items such as avocados. We want to know if the sale price and volume at which avocados are purchased reflects the state of the economy.  


#### Potential Research Questions:
1.   How does avocado purchasing relate to the housing market in different regions?
2.   How does avocado purchasing relate to average income in different regions?
3.   How do internet trends related to avocados affect the avocado market?


#### Motivation
We want to see if spending more money on avocados has a positive correlation with income. We also want to evaluate the claims that spending more money on avocados takes away from important investments like buying a house. We want to compare this data to the overall sentiment of avocados on social media platforms to determine if the claims about avocado spending relate to avocado purchasing patterns. This research can be applicable to other products that are perceived to be more or less popular among younger generations.



#### Relevant Data (Methods):
https://www.kaggle.com/timmate/avocado-prices-2020
This data set contains the prices, volumes, dates, and regions related to avocados. 

https://www.kaggle.com/alexphoffman/big-city-land-values-and-walkscores
This data set contains the walk score and land values for several big cities in the United States. 

https://www.census.gov/construction/nrs/historical_data/index.html
US Census datasets on housing sales by price range, region, and type of financing.

https://fred.stlouisfed.org/release/tables?rid=249&eid=259515#snid=259516
This webpage was used to create a data set contains the median income for each state along with D.C from the year 2019.

https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2018-01-01#
This webpage was used to create a data set contains the median income for each state along with D.C from the year 2018.

https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2017-01-01#
This webpage was used to create a data set contains the median income for each state along with D.C from the year 2017.

https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2016-01-01#
This webpage was used to create a data set contains the median income for each state along with D.C from the year 2016.

https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2015-01-01#
This webpage was used to create a data set contains the median income for each state along with D.C from the year 2015.



Other relevant data would include tweets where hastags related to avocados are used which we would scrape from twitter. 


## Data Acquisition and Processing

### Between our four members, we found datasets on different economic indicators and avocado trends with the hopes of finding connections between them later on in the project. Paolo worked with an avocado dataset and found quantitative trends while Ryland found more qualitative trends through Twitter web scraping. Somya was able to web scrape income data from years corresponding to the avocado dataset and Shannon was able to combine multiple datasets on housing sales by price and region.

### The acquisition and processing of this data covers all three requirements. Somya and Ryland met requirement 1 with web scraping, Paolo and Shannon met requirement 2 by having datasets large enough to sufficiently answer questions, and Shannon and Somya met requirement 3 by combining multiple datasets to get a more unique perspective of their topic.

### With this collected data, we hope to find correlations between economic trends and avocado trends in the next phase through things like sentiment analysis, clustering, and regression.

### Dataset 1: Avocado Prices (Paolo Fermin)

#### This data was found already in csv form through the popular dataset website Kaggle. It covers the years 2015-2020 and contains data on avocado purchasing volume and pricing within the United States. Some attributes are not likely to be helpful in the analysis (such as the specific id number of the avocado, or the size of the bag that they were puchased as), but I elected not to remove them just in case they were relevant. 

#### The "cleaning" that was necessary for this dataset did not have to do with messy data. There were no NaN values or strings where there should be numerical values. Instead, the data that is too specific. The "Geography" column contains very fine-grained detail of precisely what location the avocado purchases were made. I created a dictionary that maps specific location to a set of regions, corresponding to the U.S. Census regions that the Housing Data was given in.

In [ ]:
import pandas as pd
import os

In [ ]:
avocados = pd.read_csv('avocado-updated-2020.csv')
avocados.dtypes()

FileNotFoundError: ignored

In [ ]:
# sort geography into regions
# regions are based on U.S. census distributions

region_mapping = {
    'Northeast': ['Albany', 'Boston', 'Buffalo/Rochester', 'Harrisburg/Scranton', 
                      'Hartford/Springfield', 'New York', 'Northeast', 'Northern New England', 'Philadelphia', 'Pittsburgh', 
                     'Syracuse', ],
    
    'Midwest': ['Chicago', 'Cincinnati/Dayton', 'Columbus', 'Detroit', 'Grand Rapids', 'Great Lakes', 'Indianapolis', 'Plains',
                   'St. Louis'],
    
    'South': ['Atlanta', 'Baltimore/Washington', 'Charlotte', 'Dallas/Ft. Worth', 'Houston', 'Jacksonville', 'Louisville', 
              'Miami/Ft. Lauderdale', 'Midsouth', 'Nashville', 'New Orleans/Mobile', 'Orlando', 'Raleigh/Greensboro', 'Richmond/Norfolk',
                 'Roanoke', 'South Carolina', 'South Central', 'Southeast', 'Tampa', ],
    
    'West': ['Boise', 'California', 'Denver', 'Las Vegas', 'Los Angeles', 'Phoenix/Tucson', 'Portland', 'Sacramento', 'San Diego', 
                'San Francisco', 'Seattle', 'Spokane', 'West', 'West Tex/New Mexico'],
    
    'US': ['Total U.S.']
}

# "flatten" the dictionary so that each location has it's own separate key:value pair (corresponding to location:region)
flat_regions = {val:key for key, lst in region_mapping.items() for val in lst}
flat_regions

avocados['Region'] = avocados['geography'].map(flat_regions)
avocados.head()

In [ ]:
# write the avocado data to csv
avocados.to_csv(os.path.join(os.getcwd(), 'clean_avocados.csv'))

### Dataset 2: Income Data (Somya Jain)

#### I went on the FRED website and scraped information on median income for states for the years 2015 to 2019 since those are the years covered by the Avocado dataset. The incomes and state names were stored in a table so I searched for 'td' tags and cleaned the values before putting them into a list. I used those lists to create a dataframe which was then written to a csv file.

#### This uses the first and third data requirements since I scraped information from multiple pages and put them in the same csv.


In [ ]:
## importing libraries
import requests
import bs4
import pandas as pd

In [ ]:
def get_incomes(soup):
    '''
    Scrape FRED webpage for data on income in a given year.
    
    params:
        soup - soup object that needs to be parsed through
    return:
        list_income - list of the incomes found on the page
    '''
    
    list_income = []
    s = soup.find_all('td', class_ = 'fred-rls-elm-vl-td')

    ## creating a list of mediam income in each state from 2019
    temp = []
    n = 3
    
    ## gets only income for year on page
    for i, item in enumerate(s):
        if i%n == 0:
            temp.append(item.text)

    ## cleans value and appends it to list as an int
    for item in temp[1:]:
        num = []
        for char in item:
            if char.isdigit():
                num.append(char)
        income = ''.join(num)
        list_income.append(int(income))
    
    return list_income

In [ ]:
## getting page information for 2019 incomes
url_2019 = '''https://fred.stlouisfed.org/release/tables?rid=249&eid=259515#snid=259516'''
soup_2019 = bs4.BeautifulSoup(requests.get(url_2019).text, 'html5lib')
list_income_2019 = get_incomes(soup_2019)

## getting page information for 2018 incomes
url_2018 = '''https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2018-01-01#'''
soup_2018 = bs4.BeautifulSoup(requests.get(url_2018).text, 'html5lib')
list_income_2018 = get_incomes(soup_2018)

## getting page information for 2017 incomes
url_2017 = '''https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2017-01-01#'''
soup_2017 = bs4.BeautifulSoup(requests.get(url_2017).text, 'html5lib')
list_income_2017 = get_incomes(soup_2017)

## getting page information for 2016 incomes
url_2016 = '''https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2016-01-01#'''
soup_2016 = bs4.BeautifulSoup(requests.get(url_2016).text, 'html5lib')
list_income_2016 = get_incomes(soup_2016)

## getting page information for 2015 incomes
url_2015 = '''https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=2015-01-01#'''
soup_2015 = bs4.BeautifulSoup(requests.get(url_2015).text, 'html5lib')
list_income_2015 = get_incomes(soup_2015)

In [ ]:
## gets the list of states that income is being found for
list_states = []
states_soup = soup_2019.find_all('span', class_ = 'fred-rls-elm-nm')
[list_states.append(state.text) for state in states_soup[1:]]
print()

In [ ]:
## puts data into data frame
df = pd.DataFrame({
    
    'State': list_states,
    '2019 Median Income': list_income_2019,
    '2018 Median Income': list_income_2018,
    '2017 Median Income': list_income_2017,
    '2016 Median Income': list_income_2016,
    '2015 Median Income': list_income_2015
})

## reads data to a csv file
#df.to_csv('IncomeDate2015to2019.csv', index=False)
df

### Dataset 3: Housing Market Data (Shannon Aikens)

#### I was able to use U.S. Census data to find housing sales by region and price. These datasets will work together with the Avocado dataset to determine spending trends for both avocados and houses based on region. The housing sales by region will also work together with the scraped internet data on avocado opinions by region. 

#### I was able to export the U.S. Census tables as csv files and format them as dataframes. The U.S. Census gave me data going all the way back to the 1960s, but I narrowed the scope of the datasets to the 2000s to match the Avocado, Income, and Internet datasets. I was able to change all data to be quantitative to prepare for future analysis techniques.

In [ ]:
import pandas as pd

regionDF = pd.read_csv('pricereg_cust.csv')
priceDF = pd.read_csv('qtrsalgrp_us_ann.csv')

display(regionDF.head(10))
display(priceDF.head(10))

In [ ]:
#getting relevant rows
regionDF = regionDF.iloc[4:,:]

#creating column names
columns = ['Year','UnitedStatesMedian','NortheastMedian','MidwestMedian','SouthMedian','WestMedian',
          'UnitedStatesAverage', 'NortheastAverage', 'MidwestAverage','SouthAverage','WestAverage']
regionDF.columns = columns

#resetting index
regionDF = regionDF.reset_index(drop=True)

#cleaning data values

#looping through quantitative columns
for column in columns:
    
    #replacing commas in each column with empty string
    for i in range(len(regionDF)):
        regionDF.loc[i,column] = regionDF.loc[i,column].replace(',','')
        
    #changing type of column to integer
    regionDF = regionDF.astype({column:'int64'})
        

#narrowing years to 2002-2020 to match price dataset
regionDF = regionDF.iloc[39:]
regionDF = regionDF.reset_index(drop=True)
                            
display(regionDF)

#exporting cleaned dataframe to csv
#regionDF.to_csv(r'C:\Users\shana\OneDrive\Documents\Sophomore Year-Semester 2\Data Visualization\housing_sales_region.csv')

In [ ]:
#getting relevant rows
priceDF = priceDF.iloc[7:,:]

#creating column names
columns = ['Year','Total','Under 125,000','125,000 to 149,999','150,000 to 199,999','200,000 to 249,999',
          '250,000 to 299,999', '300,000 to 399,999', '400,000 to 499,999','500,000 to 749,999','Over 750,000']
priceDF.columns = columns

#resetting index
priceDF = priceDF.reset_index(drop=True)

#cleaning data values

#looping through quantitative columns
for column in columns:
    
    #replacing commas in each column with empty string
    for i in range(len(priceDF)):
        priceDF.loc[i,column] = priceDF.loc[i,column].replace(',','')
        
    #changing type of column to integer
    priceDF = priceDF.astype({column:'int64'})
                            
display(priceDF)

#exporting cleaned dataframe to csv
#priceDF.to_csv(r'C:\Users\shana\OneDrive\Documents\Sophomore Year-Semester 2\Data Visualization\housing_sales_price')

### Dataset 4: Web Scraping Twitter Mentions of Avocados (Ryland Hanson)

#### In order to get around the twitter developer account requirement along with limits set by the API, I decided to use a library, twint, instead. Tweets mentioning avocados are scraped from various locations that we have economic data on over a large range of time to see how the sentiment of avocados in these areas change over time and relate to the ecnonomy.

#### Twint and nest_asyncio must be installed. Documentation with download instructions are below:
https://github.com/twintproject/twint  
https://pypi.org/project/nest-asyncio/

Running the install commands below then restarting the Runtime should allow the twint module to be downloaded and working. The install commands need only be run the first time.


In [ ]:
!pip install --user --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint
#!pip3 install nest_asyncio

In [ ]:
import pandas as pd
import twint
import nest_asyncio
import re
import nltk
from nltk.corpus import stopwords
# Un-comment below line to download the full stopword list. 
#nltk.download('stopwords') 
nest_asyncio.apply()
stop_words = set(stopwords.words('english'))

In [ ]:
import string
string.punctuation, string.digits

## Function found on StackOverflow to allow the removal of emoji's through the use of regex
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

## Cleans tweets of emojis, punctuation, digits, then splits up into the bags of words and adds column to df:
def clean_tweets(dataframe):
    cleaned = dataframe['tweet'].str.replace('http\S+|www.\S+', '', case=False) # Removes URLS
    cleaned = cleaned.map(lambda tweet: remove_emoji(tweet)) # Removes emojis
    cleaned = cleaned.map(lambda txt: txt.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))) #Removes punc.
    cleaned = cleaned.map(lambda txt: txt.translate(str.maketrans(string.digits, ' '*len(string.digits)))) # Removes digits
    bag = cleaned.map(lambda txt: txt.lower().strip().split()) # Splits the words into the different bags
    bag = bag.map(lambda word_tokens: [w for w in word_tokens if not w in stop_words]) # Remove stop words

    dataframe['bag'] = bag
    return dataframe

In [ ]:
# Creates df to hold all tweets over the different regions. 
tweets_df = pd.DataFrame(columns = ['date', 'place', 'username', 'tweet', 'nlikes', 'city'])

# Loop over a list of cities to scrape tweets with geo stamps of the target city
for city in ['new york', 'los angeles', 'chicago', 'houston', 'miami', 'atlanta', 'charlotte']:
    # Configure the tweet searching. Attributes to be tweaked.
    c = twint.Config()
    c.Search = ['avocado']
    #c.Popular_tweets = False
    #c.Min_likes = 1
    c.Pandas=True
    c.Lang = 'en'
    c.Near = city
    c.Since = '2015-01-01'
    c.Until = '2019-12-30'
    c.Hide_output = True
    # Runs the search using the configuration above
    twint.run.Search(c)
    # Grabs the df made from twint, adds a target city column, then appends it to the overall df.
    temp_df = twint.storage.panda.Tweets_df[['date', 'place', 'username', 'tweet', 'nlikes']]
    temp_df['city'] = city
    tweets_df = tweets_df.append(temp_df, ignore_index = True)

In [ ]:
# Clean the tweets, remove the stop words, then set equal to cleaned_tweets
cleaned_tweets = clean_tweets(tweets_df)

## Analysis

### Inelastic vs. Elastic Goods (Paolo Fermin)

#### One of the biggest motivations for this question is whether or not young people are spending too much on avocados, which are often seen as an expensive good. To test this theory, we can look at the "price elasticity of demand", an economic concept that essentially asks how the demand changes based upon the price. 

#### The demand for an "elastic" good changes as the price changes. For example, fewer people will buy the same good when it becomes more expensive. Intuitively, most goods are elastic. However, some goods such as water and gasoline are "inelastic" - people will continue to buy those goods, no matter what the price is. The common narrative of the media suggests that young people see avocados as inelastic goods, and will continue to buy them at exorbitant prices because they don't know any better. 

#### One can determine the elasticity of a good by plotting the price vs. volume of a good. If there is a highly negative correlation between the two, then the good is elastic - as price rises, volume falls. Conversely, if there is a low correlation between the two, then the good can be considered inelastic. 

In [ ]:
# one of the first things to note is the split between conventional and organic avocados
# simply plotting the mean price of conventional avocados vs organic avocados should demonstrate this
conventional = avocados[(avocados.type == 'conventional')]
us_conv = conventional.loc[conventional.geography == 'Total U.S.', :]
organic = avocados[(avocados.type == 'organic')]
us_org = organic.loc[organic.geography == 'Total U.S.', :]

bar_df = pd.DataFrame({'Avocado Type': ['Conventional', 'Organic'], 'Average Price': [conventional.average_price.mean(), organic.average_price.mean()]})
ax = bar_df.plot.bar(x='Avocado Type', y='Average Price')

for p in ax.patches:
    ax.annotate(str(f'{p.get_height():0.02f}'), (p.get_x() * 1.05, p.get_height() * 1.005), ha='center')

#### This simple bar graph shows that there are vast differences in price between different types of avocados. In order to perform a reasonable economic analysis, only one of the two types should be picked. 

#### Let us look at organic avocados specifically, since they can be considered more of a luxury good. If organic avocados are inelastic, then conventional avocados would most certainly be inelastic as well. 

In [ ]:
# group organic data by geography and sort by highest correlation between average_price and total_volume
org_geo_groups = organic.groupby('geography')
corrs = []
for geo, df in org_geo_groups:
    corrs.append(df.average_price.corr(df.total_volume))
corr_df = pd.DataFrame({'geography':org_geo_groups.groups.keys(), 'correlation':corrs})
corr_df.sort_values('correlation', inplace=True)
corr_df

In [ ]:
fig, axs = plt.subplots(11, 5, figsize=(15, 30), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace=.5, wspace=.1)

axs = axs.ravel()

for i, (geo, corr) in enumerate(zip(corr_df.geography, corr_df.correlation)):
    # plot a scatter plot price vs. volume for each location
    df = org_geo_groups.get_group(geo)
    df.plot.scatter(x='average_price', y='total_volume', ax=axs[i])
    axs[i].ticklabel_format(axis="both", style="sci", scilimits=(0,0))
    axs[i].set_title(geo)
    
    # plot the regression line 
    fit = np.polyfit(df.average_price, df.total_volume, 1)
    f = np.poly1d(fit)
    df.insert(len(df.columns), 'Regression', f(df.average_price))
    df.plot(x='average_price', y='Regression', ax=axs[i], color='Red', legend=False)
    
    # list the actual regression
    axs[i].text(0.9, 0.8, f'R2:{corr:.2f}', verticalalignment='bottom', horizontalalignment='right', \
                transform=axs[i].transAxes, color='Red')
    
    # put a ylabel only on the endpoints
    if i % 5 != 0:
        ylabel = axs[i].axes.get_yaxis().get_label()
        ylabel.set_visible(False)

### Observations

The demand for organic avocados is correlated to the price in some geographic locations in the country, but not others. 

Could this relate to the median income in these areas? Or perhaps sentiment analysis can find a correlation? 

### Income vs. Price Analysis of Avocados (Somya Jain)

#### During my initial exploration of the data I found that avocado prices in all regions seem to follow similar trends. I wanted to figure out if changes in income affected those trends. 

In [ ]:
# Reading in clean income data
incomes = pd.read_csv('IncomeDate2015to2019.csv')

# Reading in clean avocado data
avocados = pd.read_csv('cleaned_avocados.csv')

Need to add region to income data so it corresponds to the correct avocado data
Regions are based on U.S. census distributions

In [ ]:
# adding regions to income data

# dictionary of states in each region
region_mapping = {
    'Northeast': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania' ],
    
    'Midwest': ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'],
    
    'South': ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'District of Columbia', 'West Virginia','Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'],
    
    'West': ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming', 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington'],

    'US': ['US']
}

# adding Region column to incomes dataset
incomes['Region'] = [0 for i in range(len(incomes))]
index = 0

# adding Region to rows in incomes dataset depeding on state
for state in incomes['State']:
    for region in region_mapping:
        if state in region_mapping[region]:
            incomes['Region'][index] = region
            index += 1

In [ ]:
# adding Year Column to avocados dataset
years = [int(avocados['date'][index][-4:]) for index in range(len(avocados['date']))]
avocados['Year'] = years

# grouping avocados data by year and region
avocados_grouped = avocados.groupby(['Year', 'Region']).mean()

# formatting data into lists for ease of graphing
y_midwest = [avocados_grouped['average_price'][i] for i in range(len(avocados_grouped)) if i%5 == 0 ][:-1]
y_northeast = [avocados_grouped['average_price'][i] for i in range(len(avocados_grouped)) if (i-1)%5 == 0 ][:-1]
y_south = [avocados_grouped['average_price'][i] for i in range(len(avocados_grouped)) if (i-2)%5 == 0 ][:-1]
y_west = [avocados_grouped['average_price'][i] for i in range(len(avocados_grouped)) if (i-4)%5 == 0 ][:-1]

xs = [2015, 2016, 2017, 2018, 2019]

# plotting avocados prices by year
plt.figure(figsize=(10, 10))
plt.plot(xs, y_midwest, color = 'red', label = 'Midwest')
plt.plot(xs, y_northeast, color = 'green', label = 'Northeast')
plt.plot(xs, y_south, color = 'blue', label = 'South')
plt.plot(xs, y_west, color = 'pink', label = 'West')

plt.legend(loc = 'upper left')

plt.xlabel('Years')
plt.ylabel('Average Avocado Prices')
plt.title('Average Avocado Prices by Year')
fig_prices = plt.gcf()

Avocado Prices seem to have peeked for all regions around 2017.

Observations:
Avocado prices in the Northeast were higher than any other region until around 2018.
Around 2018, the West had higher avocado prices
The prices seem to follow similar trends in all regions

Can the incomes in those regions explain some of these observations?

In [ ]:
# grouping incomes by region
incomes_grouped = incomes.groupby('Region').mean()

# formatting data into lists for ease of graphing
x_midwest = [incomes_grouped.iloc[0][i] for i in range(len(incomes_grouped.iloc[0]))]
x_midwest.reverse()

x_northeast = [incomes_grouped.iloc[1][i] for i in range(len(incomes_grouped.iloc[1]))]
x_northeast.reverse()

x_south = [incomes_grouped.iloc[2][i] for i in range(len(incomes_grouped.iloc[2]))]
x_south.reverse()

x_west = [incomes_grouped.iloc[3][i] for i in range(len(incomes_grouped.iloc[2]))]
x_west.reverse()

# plotting avocado prices vs income
plt.figure(figsize=(10, 10))
plt.scatter(x_midwest, y_midwest, color = 'red', label = 'Midwest')
plt.scatter(x_northeast, y_northeast, color = 'green', label = 'Northeast')
plt.scatter(x_south, y_south, color = 'blue', label = 'South')
plt.scatter(x_west, y_west, color = 'pink', label = 'West')

plt.legend(loc = 'upper left')

plt.xlabel('Average Median Income')
plt.ylabel('Average Avocado Prices')
plt.title('Average Avocado Prices vs. Average Median Income')
fig_incomes = plt.gcf()

Observations:
It seems as average median income increases the price of avocados increases. This does not appear to be a strong relationship. It also seems that the northeast generally has a higher average income which could account for prices being higher as well.

In [ ]:
# combining all income lists
x = []
[x.append(i) for i in x_midwest]
[x.append(i) for i in x_northeast]
[x.append(i) for i in x_south]
[x.append(i) for i in x_west]

# combining all avocado price lists
y = []
[y.append(i) for i in y_midwest]
[y.append(i) for i in y_northeast]
[y.append(i) for i in y_south]
[y.append(i) for i in y_west]

incomes_prices = pd.DataFrame(
    {
        'incomes': x,
        'prices': y
    }
)

# getting correlation coefficient for prices and incomes
corr = incomes_prices.incomes.corr(incomes_prices.prices)

The overall correlation between the average median incomes and the average prices is about 0.53. This indicates that there is a positive correlation, but it is not strong.

### Geographic Analysis of Housing Market/Income And Avocado Prices (Shannon Aikens)

#### Since the avocado, income, and housing market datasets all include regional data, I wanted to look at how similar the regions were based on the data we had collected. To do this, I used MDS with Manhattan Distances and plotted the regions to see how similar they were on a two-dimensional plane. I used MDS on a dataframe with avocado and housing market data, avocado and income data, and then all three datasets combined. The goal was to see if there was a trend among all three plots created or if any one factor (housing market or income) drastically changed the similarities between regions.

In [ ]:
### MDS on Avocado and Housing Market

#grouping avocados dataset by region
avocados_grouped = avocados.groupby('Region')

#getting the mean price and volume for avocados
price_averages_region = avocados_grouped.mean()['average_price'].reset_index()
volume_averages_region = avocados_grouped.mean()['total_volume'].reset_index()

#getting the median price and volume for avocados
price_median_region = avocados_grouped.median()['average_price'].reset_index()
volume_median_region = avocados_grouped.median()['total_volume'].reset_index()

#creating regions dataframe that will be used to conduct MDS
regionsDF = pd.DataFrame.from_dict({'Region': price_averages_region['Region'].tolist(), 
            'AvocadoMeanPrice': price_averages_region['average_price'],
            'AvocadoMeanVolume': volume_averages_region['total_volume'], 
            'AvocadoMedianPrice': price_median_region['average_price'],
            'AvocadoMedianVolume': volume_median_region['total_volume']})


#############################################################################################################

#getting quantitative data from housing price by region dataset
regionDF_averages = regionDF.iloc[13:19,1:].mean().reset_index()
regionDF_averages.columns = ['Region','CentralValue']

#getting all mean and median values for the regions
regionDF_medians = regionDF_averages.iloc[:5,:].sort_values('Region')
regionDF_means = regionDF_averages.iloc[5:10,:].sort_values('Region')

#converting mean and median values to lists to use in dataframe
region_medians = regionDF_medians['CentralValue'].tolist()
region_means = regionDF_means['CentralValue'].tolist()

#adding housing mean and median prices for each region to dataframe
regionsDF['HousingMeanPrice'] = region_means
regionsDF['HousingMedianPrice'] = region_medians

#reformatting the dataframe so it is indexed by region
regionsDF.index = regionsDF['Region']
regionsDF = regionsDF.iloc[:,1:]

display(regionsDF)

In [ ]:
import sklearn.metrics.pairwise as pairs
import sklearn.manifold as mani
import matplotlib.pyplot as plt

#Normalization
norm = (regionsDF - regionsDF.mean())/regionsDF.std()

#L1 Distance Matrix
normHD = pairs.manhattan_distances(norm)
norm = pd.DataFrame(normHD, columns=norm.index, index = norm.index)
norm #high dimensional distances for every region

#MDS calculations
mds = mani.MDS(dissimilarity='precomputed', n_init=10, max_iter=1000)
data2D = mds.fit_transform(norm)
data2D = pd.DataFrame(data2D, columns=['x','y'], index=norm.index)
data2D #suggested coordinates for plotting in two dimensions

#Plotting coordinate suggestions
ax = data2D.plot.scatter(x='x',y='y')
for i in range(len(data2D)):
    ax.text(data2D.x[i]+.2, data2D.y[i]-.1, data2D.index[i])
plt.show()

In [ ]:
### MDS on Avocado and Income Data

#copying original income dataframe
incomeDF = df.copy()

#mapping states to regions
regions_dict = {'Midwest': ['Illinois','Indiana','Iowa','Kansas','Michigan','Minnesota','Nebraska','North Dakota','Ohio',
                           'South Dakota','Wisconsin'],
               'Northeast': ['Connecticut','Delaware','Maine','Massachusetts','New Hampshire','New Jersey','New York',
                            'Pennsylvania','Rhode Island','Vermont'],
               'South': ['Alabama','Arkansas','District of Columbia','Florida','Georgia','Kentucky','Louisiana','Maryland',
                        'Mississippi','Missouri','North Carolina','Oklahoma','South Carolina','Tennessee','Texas','Virginia',
                        'West Virginia'],
               'West': ['Alaska','Arizona','California','Colorado','Hawaii','Idaho','Montana','Nevada','New Mexico','Oregon',
                       'Utah','Washington','Wyoming']}

#flattening out dictionary to map each region to each individual state
flat_regions = {val:key for key, lst in regions_dict.items() for val in lst}

#adding regions to new income dataframe
incomeDF['Region'] = incomeDF['State'].map(flat_regions)

#calculating US average (since that data is not automatically included in the income dataset)
US_average = incomeDF.iloc[:,1:-1].mean().mean()

#grouping income data by region
income_grouped = incomeDF.groupby('Region')
income_median_averages = income_grouped.mean().reset_index()

#getting median values for each region in the income dataset
income_median_averages.index = income_median_averages['Region']
income_median_averages = income_median_averages.iloc[:,1:].transpose()
income_list = income_median_averages.mean().tolist()
income_region_list = [income_list[0],income_list[1],US_average,income_list[2],income_list[3]]

#adding income data to regions dataframe and taking away housing data
regionsDF = regionsDF.iloc[:,:-2]
regionsDF['IncomeMedian'] = income_region_list

display(regionsDF)

In [ ]:
#Normalization
norm = (regionsDF - regionsDF.mean())/regionsDF.std()

#L1 Distance Matrix
normHD = pairs.manhattan_distances(norm)
norm = pd.DataFrame(normHD, columns=norm.index, index = norm.index)
norm #high dimensional distances for every region

#MDS calculations
mds = mani.MDS(dissimilarity='precomputed', n_init=10, max_iter=1000)
data2D = mds.fit_transform(norm)
data2D = pd.DataFrame(data2D, columns=['x','y'], index=norm.index)
data2D #suggested coordinates for plotting in two dimensions

#Plotting coordinate suggestions
ax = data2D.plot.scatter(x='x',y='y')
for i in range(len(data2D)):
    ax.text(data2D.x[i]+.2, data2D.y[i]-.1, data2D.index[i])
plt.show()

In [ ]:
### MDS on Avocado, Housing, and Income Data

#adding back in housing data
regionsDF['HousingMeanPrice'] = region_means
regionsDF['HousingMedianPrice'] = region_medians

display(regionsDF)

In [ ]:
#Normalization
norm = (regionsDF - regionsDF.mean())/regionsDF.std()

#L1 Distance Matrix
normHD = pairs.manhattan_distances(norm)
norm = pd.DataFrame(normHD, columns=norm.index, index = norm.index)
norm #high dimensional distances for every region

#MDS calculations
mds = mani.MDS(dissimilarity='precomputed', n_init=10, max_iter=1000)
data2D = mds.fit_transform(norm)
data2D = pd.DataFrame(data2D, columns=['x','y'], index=norm.index)
data2D #suggested coordinates for plotting in two dimensions

#Plotting coordinate suggestions
ax = data2D.plot.scatter(x='x',y='y')
for i in range(len(data2D)):
    ax.text(data2D.x[i]+.2, data2D.y[i]-.1, data2D.index[i])
plt.show()

### Sentiment Analysis of Avocado Tweets (Ryland Hanson)

One interesting method of analyzing this twitter data would be to track the sentiment of the tweets and group them by locations over time. Ffirst we can create a bag of words from each tweet, clean them up, remove stopwords, then using a list of positive and negative words we can compute a value for the sentiment. In this case, positive sentement is classified as a sentiment value at least 0.1, a negative sentement would be less than -0.1, with the rest being considered neutral. We can then visualize this by breaking up into the cities and seeing the distribution of sentiment, we can group it by time and see how sentiment changes over time, and we can combine them in many possible ways to show various information about the sentiment in the various locations over the 5 year period.

In [ ]:
## Adds a length of bag feature to the DF
cleaned_tweets['Length'] = cleaned_tweets['bag'].map(len)

## Our list of positive and negative words to test the sentiment
positives = set(open('positive.txt').read().split()) 
negatives = set(open('negative.txt').read().split())

## Calculates the sentiment value for each tweet in the DF
pos_bag = cleaned_tweets.bag.map(lambda bag: [word for word in bag if word in positives])
neg_bag = cleaned_tweets.bag.map(lambda bag: [word for word in bag if word in negatives])
cleaned_tweets['sentiment_value'] = (pos_bag.map(len) - neg_bag.map(len)) / cleaned_tweets.Length 

## Assign a sentiment grouping based on the value
cleaned['sentiment'] = cleaned.sentiment_value.map(lambda x: 'negative' if x < -0.1 else 'positive' if x > 0.1 else 'neutral')

NameError: ignored

In [ ]:
## Seperate the tweets by year to visualize
tweets_2015 = cleaned_tweets[cleaned_tweets.date.str.startswith('2015')]
tweets_2016 = cleaned_tweets[cleaned_tweets.date.str.startswith('2016')]
tweets_2017 = cleaned_tweets[cleaned_tweets.date.str.startswith('2017')]
tweets_2018 = cleaned_tweets[cleaned_tweets.date.str.startswith('2018')]
tweets_2019 = cleaned_tweets[cleaned_tweets.date.str.startswith('2019')]

tweets_2015

In [ ]:
## groupby objects for each year to plot in bar graphs
sentiment_2015 = tweets_2015.groupby(['city', 'sentiment']).sentiment.count().unstack()
sentiment_2016 = tweets_2016.groupby(['city', 'sentiment']).sentiment.count().unstack()
sentiment_2017 = tweets_2017.groupby(['city', 'sentiment']).sentiment.count().unstack()
sentiment_2018 = tweets_2018.groupby(['city', 'sentiment']).sentiment.count().unstack()
sentiment_2019 = tweets_2019.groupby(['city', 'sentiment']).sentiment.count().unstack()

## Set up the plotting. Note 2019 will go on seperate row due to odd number of plots
fig, axs = plt.subplots(2, 2, figsize = (10, 8), sharey=True)

## Plot the bar charts
sentiment_2015.plot(kind="bar", ax = axs[0, 0], title = '2015')
sentiment_2016.plot(kind="bar", ax = axs[0, 1], title = '2016')
sentiment_2017.plot(kind="bar", ax = axs[1, 0], title = '2017')
sentiment_2018.plot(kind="bar", ax = axs[1, 1], title = '2018')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()
    
## Plot the final year
sentiment_2019.plot(kind="bar", title = '2019', figsize=(5, 4))

In [ ]:
## Another visualization, while a bit scattered on one plot, can be given to show the change of sentiment over time
cleaned_tweets['day'] = cleaned_tweets.date.map(lambda date: date.split()[0]) ## Add a column for the day, to generarilize by day instead of exact time
ts_df = cleaned_tweets.groupby(['day', 'city']).sentiment_value.mean().unstack() ## Group by to get a df with sentiment values by day and city
ax = ts_df.iloc[1:, :].plot(figsize = (10, 8))
plt.axhline(y=0.1, color='black', linestyle='--', label = 'Upper bound of neutral') ## Plot the threshold for positive sentiment
plt.axhline(y=-.1, color='black', linestyle='--', label = 'Lower bound of neutral') ## Plot the threshold for negative sentiment
plt.ylabel('Sentiment Value')
plt.xlabel('Day')


## Formalized Answers from Analysis

### Write a paragraph about what important things you observed, what question(s) your analysis is answering, and how it relates to some of the other analyses performed by others.

### Paolo Fermin

#### One of the research questions we attempted to answer was whether avocados are an elastic good. This stems from the common perception in the media that avocados are inelastic, and that young people will buy them no matter the price. 

#### What we can learn from the scatterplots above is that the demand for organic avocados is elastic only in certain locations around the country. At locations where R2 values have magnitudes above 0.5, the data seems to lie close to the linear regression line. Once the magnitude of R2 falls below that threshold, the data is scattered further from the regression line. Organic avocados in these places are inelastic, and the price has little influence on the demand. 

#### Interestingly, this correlation is not seen in the aggregate "Total U.S." data - in fact, the Total U.S. correlation is worse than any particular geography. This proves that simply looking at aggregate data does not tell the whole story of this dataset. Between locations in the U.S. are vast differences in income, cost of living, and many other factors that cannot be accounted for when looking at a very high level. 

### Somya Jain

One of our research questions was "how does avocado purchasing relate to average income in different regions?" To answer this we first looked at avocado purchasing patterns between 2015 and 2019 and then analyzed the average median in the 5 regions and the prices of avocados in those same regions over different years. We found that that for all regions, the prices of avocados peaked around 2017. We also found that there is a weak positive correlation of about 0.53 between average median income and average avocado prices. This relates to the MDS analysis done on avocado and income data which found that Midwest and South were similar in their purchaing patterns. This is reflected in the scatter plot as well since the points for the Midwest and South are closer to each other. These observations are important because it shows that as incomes go up, so do the prices of avocados. This indicates that people (including millenials) can afford to buy avocados despite them being "unnecessary," which further suggests that they are not taking away from spending on more "important" purchases. 

In [ ]:
fig_prices

In [ ]:
fig_incomes

In [ ]:
print('Correlation Coefficient:',corr)

### Shannon Aikens

Building off of question 1, we also wanted to compare avocado purchasing to the housing market in different regions. Some of the assumptions made on the internet are that people who buy avocados are wasting money they could be putting towards a house, or people who are rich enough to buy avocados also are rich enough to purchase houses. To validate either of these assumptions, we needed to look at how the housing market and avocado spending habits impacted the similarity of regions. Using MDS, I found that the Midwest and South were most similar based on housing and avocado data. These regions were also closest to the US averages which gives an indication that these lower values are skewing the country averages towards lower prices and volumes. The Northeast was not only far away from the other regions but also far away from the US averages. Looking at the actual data, the Northeast has the highest avocado prices and median volume as well as the highest housing prices which supports the second claim that those who can afford avocados are those who can also afford more expensive houses. 

Since the finding that the South and Midwest are very similar regions is also a conclusion made from question 1, I wanted to add in income by region to the MDS model to see if the coordinates of the regional scatterplot shifted drastically. The South and Midwest got even closer with the income data added as factors which shows that there is a strong correlation between avocados, income, and housing as we had initially predicted. The Northeast still stood out among the rest due to its very high values in all categories. Since this general pattern was witnessed throughout all three MDS models, it shows that avocado data differs between the regions similarly to both income and housing data. The higher the income or housing prices, the higher the avocado prices and consumption will be. This shows that people will use their expendable income on avocados just as much as they will houses, so avocado purchasing isn't necessarily getting in the way of a boom in the housing market.

### Ryland Hanson

One of the initial questions was how sentiment on avacados may be in some way related to the market, be it by sale volume or price. While difficult to accurately measure the nationwide sentiment on something specific like avocados, we explored this through the use of scraping tweets that mention the word avocado from different geo stamped locations and then measuring the sentiment with a bag of words approach. What we found in this case is that the sentiment of tweets in regards to avocados are largely fairly neutral, there are a few things we can notice. When looking at how the average price for the various cities change with time, we can see that some cities exhibit a familiar behavior along with the sentiment: when prices are high, there may be more negative (or at least less positive) sentiment, but when prices lower often times the negative sentiment will as well. An example in the plot below shows New York's average price starts to go down from 2017-2018, and in the sentiment bar chart we see that while the change is quite small, especially graphically, we can see that indeed the negative sentiment went down a bit and neutral seemed to increase. In the price plot we can see that in 2019 Houston had a big jump in price, and indeed they also had a big jump in negative sentiment, despite the positive also increasing a bit. This relationship can be seen in the rest of the cities too at certain jumps in price, therefore we can indeed say there does seem to be some form of relationship between the perceived sentiment of avocados at certain locations and times, and the average pricing. This result makes sense as well, people tend to be more happy (positive) about a product when it is cheap rather than unduly expensive.

In [ ]:
## Read in the avocado pricing dataset
price_df = pd.read_csv('cleaned_avocados.csv')

## Retrieve just the organic pricing for consistency
organic = price_df[(price_df.type == 'organic')]

## Group by the year and location, then average the price for the year
gb = organic.groupby(['year', 'geography']).average_price.mean().unstack()

## Select only the cities with sentiment information
cities_avg = gb[['New York', 'Atlanta', 'Chicago', 'Houston', 'Los Angeles', 'Miami/Ft. Lauderdale']]

## Plot the average price over time for each location
cities_avg.plot(figsize = (9, 6))
plt.legend()


In [ ]:
## Compare with bar graphs created in prior section:
fig, axs = plt.subplots(2, 2, figsize = (10, 8), sharey=True)

sentiment_2015.plot(kind="bar", ax = axs[0, 0], title = '2015')
sentiment_2016.plot(kind="bar", ax = axs[0, 1], title = '2016')
sentiment_2017.plot(kind="bar", ax = axs[1, 0], title = '2017')
sentiment_2018.plot(kind="bar", ax = axs[1, 1], title = '2018')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

sentiment_2019.plot(kind="bar", title = '2019', figsize=(5, 4))
plt.show()

## Conclusion

## Shannon will whip this up last minute :)